In [ ]:
# settings for testing and running on a PC.
# Importing necessary packages:

from glob import glob
import numpy as np
import pandas as pd

import itertools

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.animation as animation
import seaborn as sns

import imageio

from matplotlib.image import NonUniformImage
from matplotlib.patches import Circle, Rectangle
from matplotlib import cm

from polyphys.visualize import plotter
from polyphys.visualize import tuner
from polyphys.manage.parser import (SumRuleCyl, TransFociCyl, TransFoc
from polyphys.manage import organizer
from polyphys.analyze import measurer
from polyphys.manage import utilizer


### Visualizing

In [ ]:
# Project details
project = 'SumRule'
#project = 'TransFoci'
project_details ={
    'SumRule':{
        'parser': SumRule,
        'space_pat': 'N*D*ac*',
        'hierarchy': 'N*',
        'space_hierarchy': 'N*',
        'attributes': ['space', 'ensemble_long', 'ensemble', 'nmon', 'dcyl',
                       'dcrowd','phi_c_bulk'
                      ],
        'time_varying_props': [ 'asphericityTMon', 'fsdTMon', 'gyrTMon',
                               'rfloryTMon','shapeTMon'],
        'equil_measures': [np.mean, np.var, measurer.sem],
        'equil_attributes': ['space', 'ensemble_long', 'ensemble', 'nmon',
                             'dcyl','dcrowd', 'phi_c_bulk',
                             'phi_c_bulk_round'
                            ],
        'equil_properties': ['asphericityMon-mean', 'asphericityMon-var',
                             'asphericityMon-sem', 'fsdMon-mean',
                             'fsdMon-var', 'fsdMon-sem', 'gyrMon-mean',
                             'gyrMon-var', 'gyrMon-sem', 'rfloryMon-mean',
                             'rfloryMon-var', 'rfloryMon-sem',
                             'shapeMon-mean', 'shapeMon-var', 'shapeMon-sem']
    },
    'TransFoci':{
        'parser': TransFoci,
        'space_pat': 'ns*nl*al*D*ac*',
        'hierarchy': 'eps*',
        'space_hierarchy': 'ns*',
        'attributes': ['space', 'ensemble_long', 'ensemble', 'nmon_small',
                       'nmon_large','dmon_large', 'dcyl', 'dcrowd',
                       'phi_c_bulk'
                      ],
        'time_varying_props': ['asphericityTMon', 'fsdTMon', 'gyrTMon',
                               'shapeTMon'],
        'equil_measures': [np.mean, np.var, measurer.sem],
        'equil_attributes': ['ensemble_long', 'ensemble', 'space', 'dcyl',
                             'dmon_large', 'nmon_large', 'nmon_small',
                             'dcrowd', 'phi_c_bulk', 'phi_c_bulk_round'],
        'equil_properties': ['asphericityMon-mean', 'asphericityMon-var',
                             'asphericityMon-sem', 'fsdMon-mean',
                             'fsdMon-var', 'fsdMon-sem', 'gyrMon-mean',
                             'gyrMon-var', 'gyrMon-sem', 'shapeMon-mean',
                             'shapeMon-var', 'shapeMon-sem']
    }
}

properties_labels =  {
    'SumRule': {
        'rfloryMon-norm': {
            'name': 'Flory radius',
            'symbol': r'$\frac{{R_F(\phi_c)}}{{R_{F,0}}}$',
            'color': 'firebrick'
        },
        'gyrMon-norm': {
            'name': 'Radius of gyration',
            'symbol': r'$\frac{{R_g(\phi_c)}}{{R_{g,0}}}$',
            'color':'steelblue'
        },
        'fsdMon-norm': {
            'name': 'Furthermost distance',
            'symbol': r'$\frac{{L(\phi_c)}}{{L_{0}}}$',
            'color': 'forestgreen'
        },
        'asphericityMon-norm': {
            'name': 'Asphericity',
            'symbol': r'$\frac{{\Delta(\phi_c)}}{{\Delta_{0}}}$',
            'color': 'goldenrod'
        },
        'shapeMon-norm': {
            'name': 'Shape parameter',
            'symbol': r'$\frac{{S(\phi_c)}}{{S_{0}}}$',
            'color': 'orchid'
        },
        'asphericityMon-mean': {
            'name': 'Asphericity',
            'symbol': r'$\Delta(\phi_c)$',
            'color': 'goldenrod'
        },
        'shapeMon-mean': {
            'name': 'Shape parameter',
            'symbol': r'$S(\phi_c)$',
            'color': 'orchid'
        },
        'rPhi': {
            'name': 'Radial volume fraction',
            'symbol': r'$\phi(r)$',
        },
        'zPhi': {
            'name': 'Longitudinal volume fraction',
            'symbol': r'$\phi(|z|)$'
        },
        'rRho': {
            'name': 'Radial number density',
            'symbol': r'$\rho(r)$'
        },
        'zRho': {
            'name': 'Longitudinal number density',
            'symbol': r'$\rho(|z|)$'
        },
        'rPhi-norm': {
            'name': 'Normalized radial volume fraction',
            'symbol': r'$\phi(r)$',
            'Mon': r'$\phi_m(r)$',
            'Crd': r'$\phi_c(r)$',
            'Sum': r'$\sum_i\phi_i(r)/a_i$',
            'Sum_constant': r'$\sum_c\phi_c(\infty)/a_c$'
        },
        'zPhi-norm': {
            'name': 'Normalized longitudinal volume fraction',
            'symbol': r'$\phi(|z|)$',
            'Mon': r'$\phi_m(|z|)$',
            'Crd': r'$\phi_c(|z|)$',
            'Sum': r'$\sum_i\phi_i(|z|)/a_i$',
            'Sum_constant': r'$\sum_c\phi_c(\infty)/a_c$'
        },
        'rPhi-norm-old': {
            'name': 'Normalized radial volume fraction',
            'symbol': r'$\phi(r)$',
            'Mon': r'${{\phi_m(r)}}/{{\phi_m(0)}}$',
            'Crd': r'${{\phi_c(r)}}/{{\phi_c(\infty)}}$',
            'Sum': r'${(\sum_i\phi_i(r)/a_i)}/{(\sum_i\phi_i(\infty)/a_i)}$',
            'Sum_constant': r'${(\sum_c\phi_c(\infty)/a_c)}/{(\sum_i\phi_i(\infty)/a_i)}$'
        },
        'zPhi-norm-old': {
            'name': 'Normalized longitudinal volume fraction',
            'symbol': r'$\phi(|z|)$',
            'Mon': r'${{\phi_m(|z|)}}/{{\phi_m(0)}}$',
            'Crd': r'${{\phi_c(|z|)}}/{{\phi_c(\infty)}}$',
            'Sum': r'${(\sum_i\phi_i(|z|)/a_i)}/{(\sum_i\phi_i(\infty)/a_i)}$',
            'Sum_constant': r'${(\sum_c\phi_c(\infty)/a_c)}/{(\sum_i\phi_i(\infty)/a_i)}$'
        },
        'rRho-norm': {
            'name': 'Normalized radial number density',
            'symbol': r'$\rho(r)$',
            'Mon': r'${{\rho_m(r)}}/{{\rho_m(0)}}$',
            'Crd': r'${{\rho_c(r)}}/{{\rho_c(\infty)}}$',
            'Sum': r'${(\sum_i\rho_i(r)a_i^2)}/{(\sum_i\rho_i(\infty)a_i^2)}$',
            'Sum_constant': r'${(\sum_c\rho_c(\infty)a_c^2)}//{(\sum_i\rho_i(\infty)a_i^2)}$'
        },
        'zRho-norm': {
            'name': 'Normalized radial number density',
            'symbol': r'$\rho(|z|)$',
            'Mon': r'${{\rho_m(|z|)}}/{{\rho_m(0)}}$',
            'Crd': r'${{\rho_c(|z|)}}/{{\rho_c(\infty)}}$',
            'Sum': r'${(\sum_i\rho_i(|z|)a_i^2)}/{(\sum_i\rho_i(\infty)a_i^2)}$',
            'Sum_constant': r'${(\sum_c\rho_c(\infty)a_c^2)}//{(\sum_i\rho_i(\infty)a_i^2)}$'
        }
    },
    'TransFoci':{
        'gyrMon-norm': {
            'name': 'Radius of gyration',
            'symbol': r'$\frac{R_g(\phi_c)}{R_{g,0}}$',
            'color':'steelblue'
        },
        'fsdMon-norm': {
            'name': 'Furthermost distance',
            'symbol': r'$\frac{L(\phi_c)}{L_{0}}$',
            'color': 'forestgreen'
        },
        'asphericityMon-norm': {
            'name': 'Asphericity',
            'symbol': r'$\frac{\Delta(\phi_c)}{\Delta_{0}}$',
            'color': 'goldenrod'
        },
        'shapeMon-norm': {
            'name': 'Shape parameter',
            'symbol': r'$\frac{S(\phi_c)}{S_{0}}$',
            'color': 'orchid'
        },
        'asphericityMon-mean': {
            'name': 'Asphericity',
            'symbol': r'$\Delta(\phi_c)$',
            'color': 'goldenrod'
        },
        'shapeMon-mean': {
            'name': 'Shape parameter',
            'symbol': r'$S(\phi_c)$',
            'color': 'orchid'
        },
        'bondsHistFoci-norm': {
            'name': 'Probability distribution function of direct bonds',
            'symbol': r'$\mathcal{P}(x_d)$',
            'color': 'orchid'
        },
        'clustersHistFoci-norm': {
            'name': 'Probability distribution function of clusters',
            'symbol': r'$\mathcal{P}(x_c)$',
            'color': 'orchid'
        },
        'pairDistHistFoci': {
            'name': 'Frequency distribution of pair distance',
            'symbol': r'$\mathcal{H}(n_i,n_j,\Delta n;r)$',
            'color': 'orchid'
        },
        'pairDistRdfFoci': {
            'name': 'Radial distribution function of pair distance',
            'symbol': r'$\mathcal{P}(n_i,n_j,\Delta n;r)$',
            'color': 'orchid'
        },
        'pairDistRdfGenDistAvg': {
            'name': 'Radial distribution function of pair distance averaged over Genomic distances',
            'symbol': r'$\mathcal{P}(\Delta n;r)$',
            'color': 'orchid'
        },
        'pairDistTFoci': {
            'name': 'Pair distance',
            'symbol': r'$r(t)$',
            'color': 'orchid'
        },
        'zPhi': {
            'name': 'Longitudinal volume fraction',
            'symbol': r'$\phi(|z|)$'
        },
        'rRho': {
            'name': 'Radial number density',
            'symbol': r'$\rho(r)$'
        },
        'zRho': {
            'name': 'Longitudinal number density',
            'symbol': r'$\rho(|z|)$'
        },
        'rPhi-norm': {
            'name': 'Normalized radial volume fraction',
            'symbol': r'$\phi(r)$',
            'Mon': r'$\phi_m(r)$',
            'Foci': r'$\phi_M(r)$',
            'Crd': r'$\phi_c(r)$',
            'Sum': r'$\sum_i\phi_i(r)/a_i$',
            'Sum_constant': r'$\sum_c\phi_c(\infty)/a_c$'
        },
        'zPhi-norm': {
            'name': 'Normalized longitudinal volume fraction',
            'symbol': r'$\phi(|z|)$',
            'Mon': r'$\phi_m(|z|)$',
            'Foci': r'$\phi_M(|z|)$',
            'Crd': r'$\phi_c(|z|)$',
            'Sum': r'$\sum_i\phi_i(|z|)/a_i$',
            'Sum_constant': r'$\sum_c\phi_c(\infty)/a_c$'
        },
        'rPhi-norm-old': {
            'name': 'Normalized radial volume fraction',
            'symbol': r'$\phi(r)$',
            'Mon': r'${{\phi_m(r)}}/{{\phi_m(0)}}$',
            'Foci': r'${{\phi_M(r)}}/{{\phi_M(0)}}$',
            'Crd': r'${{\phi_c(r)}}/{{\phi_c(\infty)}}$',
            'Sum': r'${(\sum_i\phi_i(r)/a_i)}/{(\sum_i\phi_i(\infty)/a_i)}$',
            'Sum_constant': r'${(\sum_c\phi_c(\infty)/a_c)}/{(\sum_i\phi_i(\infty)/a_i)}$'
        },
        'zPhi-norm-old': {
            'name': 'Normalized longitudinal volume fraction',
            'symbol': r'$\phi(|z|)$',
            'Mon': r'${{\phi_m(|z|)}}/{{\phi_m(0)}}$',
            'Foci': r'${{\phi_M(|z|)}}/{{\phi_M(0)}}$',
            'Crd': r'${{\phi_c(|z|)}}/{{\phi_c(\infty)}}$',
            'Sum': r'${(\sum_i\phi_i(|z|)/a_i)}/{(\sum_i\phi_i(\infty)/a_i)}$',
            'Sum_constant': r'${(\sum_c\phi_c(\infty)/a_c)}/{(\sum_i\phi_i(\infty)/a_i)}$'
        },
        'rRho-norm': {
            'name': 'Normalized radial number density',
            'symbol': r'$\rho(r)$',
            'Mon': r'${{\rho_m(r)}}/{{\rho_m(0)}}$',
            'Foci': r'${{\rho_M(r)}}/{{\rho_M(0)}}$',
            'Crd': r'${{\rho_c(r)}}/{{\rho_c(\infty)}}$',
            'Sum': r'${(\sum_i\rho_i(r)a_i^2)}/{(\sum_i\rho_i(\infty)a_i^2)}$',
            'Sum_constant': r'${(\sum_c\rho_c(\infty)a_c^2)}//{(\sum_i\rho_i(\infty)a_i^2)}$'
        },
        'zRho-norm': {
            'name': 'Normalized longitudinal number density',
            'symbol': r'$\rho(|z|)$',
            'Mon': r'${{\rho_m(|z|)}}/{{\rho_m(0)}}$',
            'Foci': r'${{\rho_M(|z|)}}/{{\rho_M(0)}}$',
            'Crd': r'${{\rho_c(|z|)}}/{{\rho_c(\infty)}}$',
            'Sum': r'${(\sum_i\rho_i(|z|)a_i^2)}/{(\sum_i\rho_i(\infty)a_i^2)}$',
            'Sum_constant': r'${(\sum_c\rho_c(\infty)a_c^2)}//{(\sum_i\rho_i(\infty)a_i^2)}$'
        }
    }
}
attr_labels = {
    "phi_c_bulk_round": "$\phi_c$",
    "time": "$\hat{{t}}$",
    "time_norm": "$\hat{{t}}/\hat{{t}}^{{max}}$",
    "lags": "$\hat{t}_{lags}$",
    "dmon_large": "${{a_M}}/{{a_m}}$",
    "dcrowd": "${{a_c}}/{{a_m}}$",
    "dcyl": "${{D}}/{{a_m}}$",
    "nmon_small": "$N_m$",
    "nmon_large": "$N_M$",
    'phi_c_bulk_norm': "${a\phi_c}/{a_c}$",
    'genomic_distance':  '$\Delta n$',
    'bin_center':'$r$',
    'bin_center-norm-dmon_large':'${{r}}/{{a_M}}$',
    'bin_center-norm':'${{r}}/{{r_{max}}}$',
    'bin_center-r':'$r$',
    'bin_center-norm-r':'${{2r}}/{{D}}$',
    'bin_center-recentered-norm-r':'${{(2r-a^{shift})}}/{{D}}$',
    'bin_center-dcrowd-r':'${{2r}}/{{a_c}}$',
    'bin_center-dcrowd-recentered-r':'${{(2r-a^{shift})}}/{{a_c}}$',
    'bin_center-z':'$z$',
    'bin_center-norm-z':'${{|z|}}/{{z_{max}}}$',
    'bin_center-dcrowd-z':'${{|z|}}/{{z_{max}}}$',
    'bin_center-dcrowd-recentered-z':'${{|z|}}/{{z_{max}}}$',
    'bin_center-theta':'$\theta$',
    'bin_center-dcrowd-theta':'${{\theta}}/{{\pi}}$',
    'bin_center-norm-theta':'${{\theta}}/{{\pi}}$',
    'nmon':'$N$',
    'nmon_large':'$N_M$',
    'phi_c_rescaled':'$\\frac{{a\phi_c}}{{a_c}}$',
    'confinement_rate':'$\kappa=\\frac{{a_c}}{{D-a_c}}$',
    'confinement_rate_r':'$\kappa=\\frac{{D-a_c}}{{a_c}}$',
    'dep_energy_max':'$\mathcal{F}_{dep}=\phi_c^{(bulk)}[1+{{3a_m}}{{2a_c}}]$',
    'int_energy_max':'$\mathcal{F}_{int}=\\frac{{Na\phi_c^{(bulk)}}}{{a_c}}[3a_ma_c + \\frac{{3}}{{2}}]$',
    'species': 'Type'
}
dmon_large_c_palette = ['orange', 'darkgoldenrod', 'forestgreen']

In [ ]:
# loading databases:
#analysis_db = '/Users/amirhsi/OneDrive - University of Waterloo/PhD Research/Jupyter/Datasets'
analysis_db = '/Users/amirhsi_mini/research_data/analysis/'
#analysis_db = '../../Datasets/' # relative
#project = 'TransFoci'
phase='ensAvg'
geometry='biaxial'
group = "all"
space_dbs = glob(analysis_db + project_details[project]['space_pat']+'-'+group+'-'+phase)
print(space_dbs)

In [ ]:
for space_db in space_dbs:
    hist2d_properties = sorted(list(set(['-'.join(file.split('-')[-3:]) for file in glob(space_db + '/' + project_details[project]['hierarchy'])])))
    hist2d_properties = [prop for prop in hist2d_properties if 
                         prop.startswith('xyHist') | prop.startswith('xzHist')
                         | prop.startswith('yzHist')]
    hist2d_properties = [prop for prop in hist2d_properties if prop.endswith('mean.npy')]
    cartesian_edges = sorted(list(set(['-'.join(file.split('-')[-2:]) for file in glob(space_db + '/' + project_details[project]['hierarchy'])])))
    cartesian_edges = [prop for prop in cartesian_edges if 
                       prop.startswith('xEdge') | prop.startswith('yEdge') | 
                       prop.startswith('zEdge')]
    properties = cartesian_edges + hist2d_properties
    space_ensAvgs = organizer.sort_filenames(glob(space_db + '/' + project_details[project]['hierarchy']), fmts=properties)
    ax_names = []
    for ens_avg in space_ensAvgs:
        s_info = project_details[project]['parser'](
            ens_avg[0],
            geometry=geometry,
            group=group,
            lineage='ensemble_long',
            ispath=True
        )
        xedges = np.load(ens_avg[0], allow_pickle=True)
        xcenters = (xedges[:-1] + xedges[1:]) / 2
        xcenters = xcenters / xcenters.max()
        yedges = np.load(ens_avg[1])
        ycenters = (yedges[:-1] + yedges[1:]) / 2
        ycenters = ycenters / ycenters.max()
        zedges = np.load(ens_avg[2])
        zcenters = (zedges[:-1] + zedges[1:]) / 2
        zcenters = zcenters / zcenters.max()
        hist_xy_crd = np.load(ens_avg[3])
        hist_xy_crd = hist_xy_crd / hist_xy_crd.sum()
        hist_xy_mon = np.load(ens_avg[4])
        hist_xy_mon = hist_xy_mon / hist_xy_mon.sum()
        hist_xz_crd = np.load(ens_avg[5])
        hist_xz_mon = np.load(ens_avg[6])
        hist_yz_crd = np.load(ens_avg[7])
        hist_yz_mon = np.load(ens_avg[8])
        hist_trans_mon = (hist_xz_mon + hist_yz_mon) * 0.5
        hist_trans_mon = hist_trans_mon / hist_trans_mon.sum()
        hist_trans_crd = (hist_xz_crd + hist_yz_crd ) * 0.5
        hist_trans_crd = hist_trans_crd / hist_trans_crd.sum()

        z_cut = 0.5
        z_mask = (zcenters >= -z_cut) & (zcenters <= z_cut) 
        zcenters_cut = zcenters[z_mask]
        hist_xz_crd_cut = hist_xz_crd[:,z_mask]
        hist_yz_crd_cut = hist_yz_crd[:,z_mask]
        hist_xz_mon_cut = hist_xz_mon[:,z_mask]
        hist_yz_mon_cut = hist_yz_mon[:,z_mask]
        #flar_cmap = mpl.colors.ListedColormap(sns.cm._flare_lut)
        cmap = sns.color_palette("rocket_r", as_cmap=True)
        cmap_mon = sns.color_palette("mako_r", as_cmap=True)
        rc_params = {
                'axes.facecolor': 'white',
                'mathtext.default': 'regular',
                'text.usetex': True,
                'axes.grid': False,
                'axes.grid.axis': 'both',
                'axes.grid.which': 'both'
                }
        if s_info.phi_c_bulk != 0:
            phi_c_bulk_round = utilizer.round_up_nearest(
                s_info.phi_c_bulk,0.025,3)
        else:
            phi_c_bulk_round = 0
        nmon= s_info.nmon
        dcyl = s_info.dcyl
        dcrowd= s_info.dcrowd
        ext = 'png'
        save_to = './'
        #with mpl.rc_context(rc_params):
        sns.set_theme(
                context='paper',
                style='ticks',
                font='Times New Roman',
                font_scale=1.6,
                rc=rc_params
            )
        interp = 'bilinear'
        fig = plt.figure(constrained_layout=True, figsize=(22, 11))
        fig.suptitle(fr"$N={nmon}, {{D}}/{{a_m}}={dcyl}, {{a_c}}/{{a_m}}={dcrowd}, \phi_c={phi_c_bulk_round}$", )
        colfigs = fig.subfigures(1, 2, wspace=0, width_ratios=[2, 1.2])
        # left fig
        leftfig = colfigs[0]
        axes = leftfig.subplots(ncols=1, nrows=4, sharex=True)
        # xz
        # crd
        ax = axes[0]
        rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(rectangle)
        im = NonUniformImage(
            ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1), cmap=cmap)
        im.set_data(zcenters_cut, xcenters, hist_xz_crd_cut)
        im.set(clip_path=rectangle, clip_on=True)
        ax.add_image(im)
        ax.set_xlim(-z_cut, z_cut)
        ax.set_ylim(-1.1, 1.1)
        #ax.set_xlabel(r"$z/z_{max}$")
        ax.set_ylabel(r"$2x/D$")
        ax.set_title(r"$\rho_c(x,z)$")
        # mon
        ax = axes[1]
        rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(rectangle)
        im = NonUniformImage(
            ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1),
                             cmap=cmap_mon)
        im.set_data(zcenters_cut, xcenters, hist_xz_mon_cut)
        im.set(clip_path=rectangle, clip_on=True)
        ax.add_image(im)
        ax.set_xlim(-z_cut, z_cut)
        ax.set_ylim(-1.1, 1.1)
        #ax.set_xlabel(r"$z/z_{max}$")
        ax.set_ylabel(r"$2x/D$")
        ax.set_title(r"$\rho_m(x,z)$")
        # yz
        # crd
        ax = axes[2]
        rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(rectangle)
        im = NonUniformImage(
            ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1),
                             cmap=cmap)
        im.set_data(zcenters_cut, ycenters, hist_yz_crd_cut)
        im.set(clip_path=rectangle, clip_on=True)
        ax.add_image(im)
        ax.set_xlim(-z_cut, z_cut)
        ax.set_ylim(-1.1, 1.1)
        #ax.set_xlabel(r"$z/z_{max}$")
        ax.set_ylabel(r"$2y/D$")
        ax.set_title(r"$\rho_c(y,z)$")
        # mon
        ax = axes[3]
        rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(rectangle)
        im = NonUniformImage(
            ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1),
                             cmap=cmap_mon)
        im.set_data(zcenters_cut, ycenters, hist_yz_mon_cut)
        im.set(clip_path=rectangle, clip_on=True)
        ax.add_image(im)
        ax.set_xlim(-z_cut, z_cut)
        ax.set_ylim(-1.1, 1.1)
        ax.set_xlabel(r"$z/z_{max}$")
        ax.set_ylabel(r"$2y/D$")
        ax.set_title(r"$\rho_m(y,z)$")
        # right fig
        rightfig = colfigs[1]
        axes = rightfig.subplots(ncols=1, nrows=2, sharex=True)
        # crd
        ax = axes[0]
        circle = Circle((0, 0),1, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(circle)
        im = NonUniformImage(ax, interpolation=interp, extent=(-1, 1, -1, 1),
                             cmap=cmap)
        im.set_data(xcenters, ycenters, hist_xy_crd)
        im.set(clip_path=circle, clip_on=True)
        ax.add_image(im)
        ax.set_box_aspect(1)
        ax.set_xlim(-1.05, 1.05)
        ax.set_xticks(np.arange(-1,1.25,0.25))
        ax.set_ylim(-1.05, 1.05)
        ax.set_yticks(np.arange(-1,1.25,0.25))
        #ax.set_xlabel(r"$2x/D$")
        ax.set_ylabel(r"$2y/D$")
        ax.set_title(r"$\rho_c(x,y)$")
        # mon
        ax = axes[1]
        circle = Circle((0, 0),1, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(circle)
        im = NonUniformImage(ax, interpolation=interp, extent=(-1, 1, -1, 1),
                             cmap=cmap_mon)
        data = im.set_data(xcenters, ycenters, hist_xy_mon)
        im.set(clip_path=circle, clip_on=True)
        ax.add_image(im)
        ax.set_box_aspect(1)
        ax.set_xlim(-1.05, 1.05)
        ax.set_xticks(np.arange(-1,1.25,0.25))
        ax.set_ylim(-1.05, 1.05)
        ax.set_yticks(np.arange(-1,1.25,0.25))
        ax.set_xlabel(r"$2x/D$")
        ax.set_ylabel(r"$2y/D$")
        ax.set_title(r"$\rho_m(x,y)$")
        rightfig.colorbar(
            data, shrink=0.8, ax=axes, cmap=cmap, label=r"$\rho_c(y,z)$")
        rightfig.colorbar(
            data, shrink=0.8, ax=axes, cmap=cmap_mon, label=r"$\rho_m(y,z)$")
        output = s_info.ensemble_long + '-SumRule-hist2d'
        output = save_to + output + '.' + ext
        ax_names.append(output)
        fig.savefig(output, bbox_inches='tight')
        plt.close()
    with imageio.get_writer(s_info.space+ '.gif', mode='I', fps=2) as writer:
        for filename in ax_names:
            image = imageio.imread(filename)
            writer.append_data(image)

### single 

In [ ]:

sim = 'N2000epsilon5.0r15.5lz379.5sig4.0nc3203dt0.005bdump1000adump5000ens7.j01-all'
xedges = np.load(sim + '-xEdge.npy')
yedges = np.load(sim + '-yEdge.npy')
zedges = np.load(sim + '-zEdge.npy')
xcenters = (xedges[:-1] + xedges[1:]) / 2
xcenters = xcenters / xcenters.max()
ycenters = (yedges[:-1] + yedges[1:]) / 2
ycenters = ycenters / ycenters.max()
zcenters = (zedges[:-1] + zedges[1:]) / 2
zcenters = zcenters / zcenters.max()

In [ ]:
hist_xy_mon = np.load(sim + '-xyHistMon.npy')
hist_xy_mon = hist_xy_mon / hist_xy_mon.sum()

hist_xz_mon = np.load(sim + '-xzHistMon.npy')
hist_yz_mon = np.load(sim + '-yzHistMon.npy')
hist_trans_mon = (hist_xz_mon + hist_yz_mon) * 0.5
hist_trans_mon = hist_trans_mon / hist_trans_mon.sum()

hist_xy_crd = np.load(sim + '-xyHistCrd.npy')
hist_xy_crd = hist_xy_crd / hist_xy_crd.sum()
hist_xz_crd = np.load(sim + '-xzHistCrd.npy')
hist_yz_crd = np.load(sim + '-yzHistCrd.npy')
hist_trans_crd = (hist_xz_crd + hist_yz_crd ) * 0.5
hist_trans_crd = hist_trans_crd / hist_trans_crd.sum()

In [ ]:
z_cut = 0.5
z_mask = (zcenters >= -z_cut) & (zcenters <= z_cut) 
zcenters_cut = zcenters[z_mask]
hist_xz_crd_cut = hist_xz_crd[:,z_mask]
hist_yz_crd_cut = hist_yz_crd[:,z_mask]
hist_xz_mon_cut = hist_xz_mon[:,z_mask]
hist_yz_mon_cut = hist_yz_mon[:,z_mask]

#### Sub-figure

In [ ]:

#flar_cmap = mpl.colors.ListedColormap(sns.cm._flare_lut)
cmap = sns.color_palette("rocket_r", as_cmap=True)
rc_params = {
        'axes.facecolor': 'white',
        'mathtext.default': 'regular',
        'text.usetex': True,
        'axes.grid': False,
        'axes.grid.axis': 'both',
        'axes.grid.which': 'both'
        }
phi_c_bulk_round = 0.2
nmon= 2000
dcyl = 30.0
dcrowd= 4.0
ext = 'pdf'
save_to = './'
#with mpl.rc_context(rc_params):
sns.set_theme(
        context='paper',
        style='ticks',
        font='Times New Roman',
        font_scale=1.6,
        rc=rc_params
    )
interp = 'bilinear'
fig = plt.figure(constrained_layout=True, figsize=(22, 11))
fig.suptitle(fr"$N={nmon}, {{D}}/{{a_m}}={dcyl}, {{a_c}}/{{a_m}}={dcrowd}, \phi_c={phi_c_bulk_round}$", )
colfigs = fig.subfigures(1, 2, wspace=0, width_ratios=[3, 1.2])
# left fig
leftfig = colfigs[0]
axes = leftfig.subplots(ncols=1, nrows=4, sharex=True)
# xz
# crd
ax = axes[0]
rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
            edgecolor='black', linewidth=1, alpha=1, zorder=1)
ax.add_patch(rectangle)
im = NonUniformImage(
    ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1), cmap=cmap)
im.set_data(zcenters_cut, xcenters, hist_xz_crd_cut)
im.set(clip_path=rectangle, clip_on=True)
ax.add_image(im)
ax.set_xlim(-z_cut, z_cut)
ax.set_ylim(-1.1, 1.1)
#ax.set_xlabel(r"$z/z_{max}$")
ax.set_ylabel(r"$2x/D$")
ax.set_title(r"$\rho_c(x,z)$")
# mon
ax = axes[1]
rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
            edgecolor='black', linewidth=1, alpha=1, zorder=1)
ax.add_patch(rectangle)
im = NonUniformImage(
    ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1),
                     cmap=cmap)
im.set_data(zcenters_cut, xcenters, hist_xz_mon_cut)
im.set(clip_path=rectangle, clip_on=True)
ax.add_image(im)
ax.set_xlim(-z_cut, z_cut)
ax.set_ylim(-1.1, 1.1)
#ax.set_xlabel(r"$z/z_{max}$")
ax.set_ylabel(r"$2x/D$")
ax.set_title(r"$\rho_m(x,z)$")
# yz
# crd
ax = axes[2]
rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
            edgecolor='black', linewidth=1, alpha=1, zorder=1)
ax.add_patch(rectangle)
im = NonUniformImage(
    ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1),
                     cmap=cmap)
im.set_data(zcenters_cut, ycenters, hist_yz_crd_cut)
im.set(clip_path=rectangle, clip_on=True)
ax.add_image(im)
ax.set_xlim(-z_cut, z_cut)
ax.set_ylim(-1.1, 1.1)
#ax.set_xlabel(r"$z/z_{max}$")
ax.set_ylabel(r"$2y/D$")
ax.set_title(r"$\rho_c(y,z)$")
# mon
ax = axes[3]
rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
            edgecolor='black', linewidth=1, alpha=1, zorder=1)
ax.add_patch(rectangle)
im = NonUniformImage(
    ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1),
                     cmap=cmap)
im.set_data(zcenters_cut, ycenters, hist_yz_mon_cut)
im.set(clip_path=rectangle, clip_on=True)
ax.add_image(im)
ax.set_xlim(-z_cut, z_cut)
ax.set_ylim(-1.1, 1.1)
ax.set_xlabel(r"$z/z_{max}$")
ax.set_ylabel(r"$2y/D$")
ax.set_title(r"$\rho_m(y,z)$")
# right fig
rightfig = colfigs[1]
axes = rightfig.subplots(ncols=1, nrows=2, sharex=True)
# crd
ax = axes[0]
circle = Circle((0, 0),1, facecolor='none',
            edgecolor='black', linewidth=1, alpha=1, zorder=1)
ax.add_patch(circle)
im = NonUniformImage(ax, interpolation=interp, extent=(-1, 1, -1, 1),
                     cmap=cmap)
im.set_data(xcenters, ycenters, hist_xy_crd)
im.set(clip_path=circle, clip_on=True)
ax.add_image(im)
ax.set_box_aspect(1)
ax.set_xlim(-1.05, 1.05)
ax.set_xticks(np.arange(-1,1.25,0.25))
ax.set_ylim(-1.05, 1.05)
ax.set_yticks(np.arange(-1,1.25,0.25))
#ax.set_xlabel(r"$2x/D$")
ax.set_ylabel(r"$2y/D$")
ax.set_title(r"$\rho_c(x,y)$")
# mon
ax = axes[1]
circle = Circle((0, 0),1, facecolor='none',
            edgecolor='black', linewidth=1, alpha=1, zorder=1)
ax.add_patch(circle)
im = NonUniformImage(ax, interpolation=interp, extent=(-1, 1, -1, 1),
                     cmap=cmap)
data = im.set_data(xcenters, ycenters, hist_xy_mon)
im.set(clip_path=circle, clip_on=True)
ax.add_image(im)
ax.set_box_aspect(1)
ax.set_xlim(-1.05, 1.05)
ax.set_xticks(np.arange(-1,1.25,0.25))
ax.set_ylim(-1.05, 1.05)
ax.set_yticks(np.arange(-1,1.25,0.25))
ax.set_xlabel(r"$2x/D$")
ax.set_ylabel(r"$2y/D$")
ax.set_title(r"$\rho_m(x,y)$")
rightfig.colorbar(data, shrink=0.8, ax=axes, cmap=cmap)
plt.show()
output = sim + '-SumRule-hist2d'
fig.savefig(save_to + output + '.' + ext, bbox_inches='tight')